# This is the notebook for QM9 curation in the manscript
QM9 dataset was downloaded from http://quantum-machine.org/datasets/, which uncharacterized.txt, gdb17_qm9_energy.smi, and atomref.txt in the notebook were extracted from. 

In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.notebook import tqdm
from collections import defaultdict
from pysmiles import read_smiles
from openbabel import pybel
import networkx as nx
import re
print(rdkit.__version__)

2022.03.2


In [2]:
# Canonicalize SMILES string
def canon_smi(smi):
    mol = Chem.MolFromSmiles(smi)
    Chem.SanitizeMol(mol)
    smi = Chem.MolToSmiles(mol)
    
    return smi

## 1. Remove “uncharacterized molecules” defined in the original QM9 paper.
## 2. Remove molecules that failed RDKit SMILES canonicalization from SMILES provided by the original QM9 paper after DFT optimization.

In [3]:
# Build uncharacterized molecules indicated in the original QM9 paper with canonicalized SMILES 
with open('./uncharacterized.txt','r') as uc:
    uncha = list()
    for line in uc:
        line = line.rstrip().split()
        if len(line) > 2:
            if line[0].isdigit():
                uncha.append(line[1])
len(uncha)

3054

In [4]:
# Build mol dict from QM9 with key=canon_smi, value={parsed_smi,energy}
with open('./gdb17_qm9_energy.smi','r') as qm9:
    qm9_dict = defaultdict(list)
    for line in tqdm(qm9, total=133885):
        try:
            line = line.rstrip().split()
            gdb_smi = line[1]
            parsed_smi = line[2]
            c_smi = canon_smi(parsed_smi)
            if gdb_smi in uncha:
                continue
            energy = float(line[3])
            qm9_dict[canon_smi(gdb_smi)].append({'parsed_smi':c_smi,'energy':energy})
        except:
            print(line)

  0%|          | 0/133885 [00:00<?, ?it/s]

[10:33:37] Explicit valence for atom # 0 N, 4, is greater than permitted
[10:33:37] Explicit valence for atom # 2 N, 4, is greater than permitted
[10:33:37] Explicit valence for atom # 2 N, 4, is greater than permitted
[10:33:37] Explicit valence for atom # 3 N, 4, is greater than permitted
[10:33:37] Explicit valence for atom # 3 N, 4, is greater than permitted
[10:33:37] Explicit valence for atom # 3 N, 4, is greater than permitted


['000271', '[NH3+]CCC([O-])=O', '[NH3]CCC(=O)[O]', '-323.582949']
['000282', 'NC(=[NH2+])C([O-])=O', 'NC(=[NH2])C(=O)[O]', '-338.530072']
['001117', 'NC(=[NH2+])CC([O-])=O', 'NC(=[NH2])[CH2].C(=O)=O', '-377.830859']
['001642', 'C[NH+]=C(N)C([O-])=O', 'CNC(=[NH2])C(=O)[O]', '-377.816466']
['001648', 'CNC(=[NH2+])C([O-])=O', 'CNC(=[NH2])C(=O)[O]', '-377.816463']
['001670', 'CC(C[NH3+])C([O-])=O', 'C[C@@H](C[NH3])C(=O)[O]', '-362.87621']


[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6
[10:33:38] Explicit valence for atom # 1 N, 4, is greater than permitted
[10:33:38] Explicit valence for atom # 0 N, 4, is greater than permitted
[10:33:38] Explicit valence for atom # 0 N, 4, is greater than permitted
[10:33:38] Explicit valence for atom # 0 N, 4, is greater than permitted
[10:33:38] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[10:33:38] Explicit valence for atom # 5 N, 4, is greater than permitted
[10:33:38] Explicit valence for atom # 3 N, 4, is greater than permitted
[10:33:38] Explicit valence for atom # 4 N, 4, is greater than permitted
[

['003828', 'N=C1C=CNN=N1', '[NH]c1cc[nH]nn1', '-335.586914']
['003832', 'N=C1C=NNN=C1', '[NH]c1cn[nH]nc1', '-335.590859']
['003833', 'N=C1C=NNN=N1', '[NH]c1cn[nH]nn1', '-351.615495']
['003842', 'N=C1NN=CC=N1', '[NH]c1[nH]nccn1', '-335.61837']
['003843', 'N=C1NN=CN=C1', '[NH]c1[nH]ncnc1', '-335.613384']
['003844', 'N=C1NN=NC=C1', '[NH]c1[nH]nncc1', '-335.591582']
['003845', 'N=C1NN=NC=N1', '[NH]c1[nH]nncn1', '-351.646368']
['004015', 'C[NH2+]CC(CO)=N[O-]', 'C[NH2]C[C](CO)[N][O]', '-418.138616']
['004018', 'NC(=[NH2+])C(=N[O-])C#N', '[NH2]=C(N)C(=N[O])C#N', '-410.814514']
['004019', 'NC(=[NH2+])C(C=O)=N[O-]', '[NH2]=C(N)C(=N[O])C=O', '-431.877352']
['004028', 'NC(=[NH2+])C(CO)=N[O-]', '[NH2]=C(N)C(=N[O])CO', '-433.071644']
['004419', 'OC1=CN=CC(=N)N1', 'Oc1cncc([NH])[nH]1', '-394.814053']
['004427', 'OC1=NC=NC(=O)O1', 'OC(=O)[N]C=[N]=C=O', '-450.668952']
['005804', 'CC(C)([NH3+])CC([O-])=O', 'CC(C)([NH3])CC(=O)[O]', '-402.176006']
['006033', 'CC(C)(C[NH3+])C([O-])=O', 'CC(C)(C[NH3])C(=O)

...

['127659', 'CN1N=CN=C(C)C1=N', 'Cn1ncnc(C)c1[NH]', '-414.202784']
['127661', 'CN1N=CN=C(N)C1=N', 'Cn1ncnc(N)c1[NH]', '-430.266048']
['127663', 'CN1N=CN=C(O)C1=N', 'Cn1ncnc(O)c1[NH]', '-450.146004']
['127686', 'CN1N=NC(=N)C(C)=N1', 'Cn1nnc([NH])c(C)n1', '-430.206734']
['127687', 'CN1N=NC(=N)C(N)=N1', 'Cn1nnc([NH])c(N)n1', '-446.271424']
['127688', 'CN1N=NC(=N)C=C1C', 'Cn1nnc([NH])cc1C', '-414.17072']
['127689', 'CN1N=NC(=N)C=C1N', 'Cn1nnc([NH])cc1N', '-430.221971']
['127690', 'CN1N=NC(=N)C=C1O', 'Cn1nnc([NH])cc1O', '-450.101747']
['127695', 'CN1N=NC(=N)NC1=O', 'Cn1nnc([NH])[nH]c1=O', '-466.197913']
['127696', 'CN1N=NC(=N)OC1=O', 'Cn1nnc([NH])oc1=O', '-486.06067']
['127723', 'CN1N=NC(C)=CC1=N', 'Cn1nnc(C)cc1[NH]', '-414.181969']
['127725', 'CN1N=NC(C)=NC1=N', 'Cn1nnc(C)nc1[NH]', '-430.238988']
['127741', 'CN1N=NC(N)=CC1=N', 'Cn1nnc(N)cc1[NH]', '-430.239106']
['127743', 'CN1N=NC(N)=NC1=N', 'Cn1nnc(N)nc1[NH]', '-446.305892']
['127752', 'CN1N=NC(O)=NC1=N', 'Cn1nnc(O)nc1[NH]', '-466.178804']

[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 8
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 8
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 8
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 8
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 8
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 8
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[10:34:09] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6
[10:34:09] Can't kekulize mo

['129023', 'N=C1C=C(NN=N1)C#C', '[NH]c1cc([nH]nn1)C#C', '-411.727865']
['129027', 'N=C1C=C2CCN2N=N1', '[NH]c1cc2CCn2nn1', '-412.941444']
['129036', 'N=C1C=CN(C=O)N=N1', '[NH]c1ccn(C=O)nn1', '-448.908481']
['129037', 'N=C1C=CN=NN1C=O', '[NH]c1ccnnn1C=O', '-448.909509']
['129056', 'N=C1C=NC=NN1C=O', '[NH]c1cncnn1C=O', '-448.934689']
['129060', 'N=C1C=NN(C=O)N=C1', '[NH]c1cnn(C=O)nc1', '-448.916513']
['129061', 'N=C1C=NN(C=O)N=N1', '[NH]c1cnn(C=O)nn1', '-464.937633']
['129063', 'N=C1C=NN2CCC2=N1', '[NH]c1cnn2CCc2n1', '-412.973447']
['129069', 'N=C1C=NNC(=O)N=C1', '[NH]c1cn[nH]c(=O)nc1', '-448.92212']
['129071', 'N=C1C=NNC=C1C#C', '[NH]c1cn[nH]cc1C#C', '-395.702435']
['129074', 'N=C1C=NNC=CC1=O', '[NH]c1cn[nH]ccc1=O', '-432.859358']
['129076', 'N=C1C=NNN=C1C#C', '[NH]c1cn[nH]nc1C#C', '-411.729079']
['129100', 'N=C1C=NON=NC1=O', '[NH]c1cnoc1=O.N#N', '-484.878482']
['129123', 'N=C1N=CC=NN1C=O', '[NH]c1nccnn1C=O', '-448.938032']
['129127', 'N=C1N=CN=NN1C=O', '[NH]c1ncnnn1C=O', '-464.961718']


['132096', 'CC1=C(F)NC(=N)C=N1', 'Cc1c(F)[nH]c([NH])cn1', '-458.128493']
['132116', 'NC1=C(F)NC(=N)C=N1', 'Nc1c(F)[nH]c([NH])cn1', '-474.173475']
['132138', 'FC1=C(F)N=CC(=N)N1', 'Fc1c(F)ncc([NH])[nH]1', '-518.071091']
['132343', 'OC1=NC(F)=NC(=O)O1', 'OC(=O)[N]C(=[N]=C=O)F', '-549.926388']
['132344', 'OC1=NC(=O)OC(F)=N1', 'OC(=[N]=C=O)[N]C(=O)F', '-549.927723']
['132345', 'FC1=NC(F)=NC(=O)O1', 'FC(=O)[N]C(=[N]=C=O)F', '-573.94274']
['132438', 'CC1=NC=C(F)NC1=N', 'Cc1ncc(F)[nH]c1[NH]', '-458.127722']
['132956', 'CN1C(=N)C=NC=C1F', 'Cn1c([NH])cncc1F', '-458.118799']
['133007', 'CN=C1NC(F)=CN=C1', 'C[N]c1[nH]c(F)cnc1', '-458.11307']
['133136', 'CC1=C(F)C(=N)N=NN1', 'Cc1c(F)c([NH])nn[nH]1', '-474.124801']
['133139', 'CC1=C(F)C(=N)NN=N1', 'Cc1c(F)c([NH])[nH]nn1', '-474.131326']
['133159', 'CC1=C(F)N=NNC1=N', 'Cc1c(F)nn[nH]c1[NH]', '-474.139031']
['133164', 'CC1=C(F)NN=NC1=N', 'Cc1c(F)[nH]nnc1[NH]', '-474.129359']
['133189', 'CC1=NC(F)=NNC1=N', 'Cc1nc(F)n[nH]c1[NH]', '-474.164708']
['133196

[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 5 7 8
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 5 7 8
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 4 5 6
[10:34:10] Explicit valence for atom # 5 N, 4, is greater than permitted
[10:34:10] Explicit valence for atom # 2 N, 4, is greater than permitted
[10:34:10] Explicit valence for atom # 5 N, 4, is greater than permitted
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 2 4 5 6 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 2 4 6 7 8
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 4 6 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 4 7 8
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 4 5 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 5 6 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms: 1 2 4 6 7
[10:34:10] Can't kekulize mol.  Unkekulized atoms

In [5]:
# Only keep the conformer with the lowest energy
for smi in tqdm(qm9_dict):
    mol_info = min(qm9_dict[smi], key=lambda k: k['energy'])
    qm9_dict[smi] = mol_info
len(qm9_dict)

  0%|          | 0/130132 [00:00<?, ?it/s]

130132

## 3. Remove molecules that failed graph isomorphism check between SMILES from GDB17 and SMILES provided by the original QM9 paper after DFT optimization.

In [6]:
# Finding molelcules having different connectivity after B3LYP optimization in the original QM9 paper
connect = list()
for gdb_smi in tqdm(qm9_dict):
    n1 = read_smiles(gdb_smi)
    nonstereo_smi = re.sub(r'\\|\/|\@','', qm9_dict[gdb_smi]['parsed_smi'])
    n2 = read_smiles(nonstereo_smi)
    if (not nx.is_isomorphic(n1,n2)):
        connect.append(gdb_smi)
connect

  0%|          | 0/130132 [00:00<?, ?it/s]

['CC1C2C3C4=CCC13C42', 'CN1C2C3C4=CCC31C42']

In [7]:
# Remove gdb_smi from qm9_dict
for smi in connect:
    qm9_dict.pop(smi, None)
len(qm9_dict)

130130

## 4. Remove molecules having atomization energy per bond smaller than 100 kcal/mol using atomic and molecular energies provided by the original QM9 paper.

In [8]:
# Build a atom energy dictionary
with open('atomref.txt','r') as f:
    atom_dict = {'H':1,'C':6,'N':7,'O':8,'F':9}
    atomref = {}
    for i in range(5):
        next(f)
    for i in range(5):
        line = f.readline().rstrip().split()
        atom_num = atom_dict[line[0]]
        atomref[atom_num] = float(line[2])
atomref

{1: -0.500273, 6: -37.846772, 7: -54.583861, 8: -75.064579, 9: -99.71873}

In [9]:
# Get atomization energy and atomization energy per bond in kcal/mol
def getAtomizationEnergy(smiles, energy, atomref):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    atom_energy_list = [atomref[atom.GetAtomicNum()] for atom in mol.GetAtoms()]
    ae = (sum(atom_energy_list) - energy) * 627.5 # Hartree to kcal/mol
    num_of_bonds = len(mol.GetBonds()) 
    ae_per_bond = ae / num_of_bonds
    
    return ae, ae_per_bond

In [10]:
# Add AE per bond into qm9_dict
for gdb_smi in tqdm(qm9_dict):
    ae, ae_per_bond = getAtomizationEnergy(qm9_dict[gdb_smi]['parsed_smi']\
                                       , qm9_dict[gdb_smi]['energy'], atomref)
    qm9_dict[gdb_smi]['AE'] = ae
    qm9_dict[gdb_smi]['AE_per_bond'] = ae_per_bond

  0%|          | 0/130130 [00:00<?, ?it/s]

In [11]:
# Get molecules with AE per bond < 100 kcal/mol
low_ae_mol = [(gdb_smi,qm9_dict[gdb_smi]['AE_per_bond']) for gdb_smi in qm9_dict if qm9_dict[gdb_smi]['AE_per_bond'] < 100]
len(low_ae_mol)

104964

In [12]:
# Remove low AE per bond molecules from qm9_dict
for mol in low_ae_mol:
    qm9_dict.pop(mol[0], None)
len(qm9_dict)

25166

## 5. Retain molecules with at least one ring.

In [13]:
# Find molecules in qm9_dict that don't have ring structure
no_ring = list()
for gdb_smi in tqdm(qm9_dict):
    mol = Chem.MolFromSmiles(gdb_smi)
    if Chem.GetSSSR(mol) == 0:
        no_ring.append(gdb_smi)
no_ring

  0%|          | 0/25166 [00:00<?, ?it/s]

['O',
 'C#C',
 'C#N',
 'C=O',
 'C#CC',
 'CC#N',
 'CC=O',
 'NC=O',
 'CC(C)=O',
 'CC(N)=O',
 'NC(N)=O',
 'C#CC#C',
 'C#CC#N',
 'N#CC#N',
 'C#CC=O',
 'N#CC=O',
 'O=CC=O',
 'CC#CC',
 'C#CCC',
 'CCC#N',
 'N#CCN',
 'C#CCO',
 'N#CCO',
 'CCC=O',
 'CNC=O',
 'COC=O',
 'O=CCO',
 'C#CC(C)=O',
 'CC(=O)C#N',
 'C#CC(N)=O',
 'CC(=O)C=O',
 'NC(=O)C=O',
 'C#CC(C)C',
 'CC(C)C#N',
 'CC(N)C#N',
 'C#CC(C)O',
 'CC(O)C#N',
 'CC(O)C=O',
 'CC(=O)CO',
 'CCC(C)=O',
 'CCC(N)=O',
 'COC(C)=O',
 'COC(N)=O',
 'NC(=O)CO',
 'C#CCC#C',
 'C#CCC#N',
 'N#CCC#N',
 'C#CCC=O',
 'N#CCC=O',
 'N=CNC=O',
 'N=COC=O',
 'O=CNC=O',
 'C#CC#CC',
 'CC#CC#N',
 'CC#CC=O',
 'CC#CCO',
 'CC#CCC',
 'C#CCCC',
 'CCCC#N',
 'CNCC#N',
 'C#CCOC',
 'COCC#N',
 'C#CCCO',
 'N#CCCO',
 'CCOC=O',
 'COCC=O',
 'O=CCCO',
 'C#CC(C)=NO',
 'FC(F)(F)F',
 'C#CC(C)(C)C',
 'CC(C)(C)C#N',
 'CC(C)(N)C#N',
 'C#CC(C)(C)O',
 'CC(C)(O)C#N',
 'CC#CC(C)=O',
 'CC#CC(N)=O',
 'CC#CC(C)C',
 'CC#CC(C)O',
 'C#CCC(C)=O',
 'CC(=O)CC#N',
 'C#CCC(N)=O',
 'N#CCC(N)=O',
 'CC(=N)NC=O',


In [14]:
# Remove no-ring molecules from qm9_dict
for smi in no_ring:
    qm9_dict.pop(smi, None)
len(qm9_dict)

17546

## 6. Remove molecules that failed the RDKit embedding method using the srETKDGv3 algorithm.

In [15]:
# Find molecules that fail for rdkit embedding
embed_fail = list()
for gdb_smi in tqdm(qm9_dict):
    mol = Chem.MolFromSmiles(gdb_smi)
    mol = Chem.AddHs(mol)
    params = Chem.rdDistGeom.srETKDGv3()
    params.randomSeed = 0xf00d
    params.clearConfs = True
    
    if (AllChem.EmbedMolecule(mol, params=params)) == -1:
        print(gdb_smi)
        embed_fail.append(gdb_smi)

  0%|          | 0/17546 [00:00<?, ?it/s]

N#CC12CCC1C(=O)O2
C#CC12COC1C(=O)O2
N#CC12COC1C(=O)O2
N#CC1CC2C(=O)OC12
C#CC1OC2C(=O)OC12
N#CC1OC2C(=O)OC12
N#CC1CC2OC(=O)C12
N#CC12CCC1OC2=O
C#CC12OCC1OC2=O
N#CC12OCC1OC2=O


In [16]:
# Remove embedding failing molecules from qm9_dict
for smi in embed_fail:
    qm9_dict.pop(smi, None)
len(qm9_dict)

17536

## 7. Remove molecules having bond(s) broken or rearrangement reactions after reorganization energy calculations described in the manuscript.

In [17]:
# This function was run after DFT optimizations were done
# origin_smi: original smiles string, geom_after_opt: geometry after DFT optimization in XYZ format
def graph_check(origin_smi, geom_after_opt):
    g1 = read_smiles(origin_smi, explicit_hydrogen=True)
    g1_noH = read_smiles(origin_smi)
    try:
        smi = pybel.readstring('xyz',geom_after_opt).write('smi').strip()
        if '.' in smi:
            return origin_smi
        g2 = read_smiles(smi, explicit_hydrogen=True)
        g2_noH = read_smiles(smi)
    except:
        return origin_smi
    
    if (not nx.is_isomorphic(g1,g2)) or (not nx.is_isomorphic(g1_noH,g2_noH, node_match=lambda n1,n2:n1['hcount']==n2['hcount'])):
        return origin_smi
    else:
        return False
    